## Shapley effects for RC and $\theta_{11}$

In [80]:
import os
import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp
from functools import partial
from ruspy.estimation.estimation import estimate
#from ruspy.simulation.simulation import simulate
from ruspy.model_code.fix_point_alg import calc_fixp
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.estimation.estimation_transitions import create_transition_matrix
from ruspy.model_code.demand_function import get_demand 
from python.econsa_shapley import get_shapley
from python.econsa_shapley import _r_condmvn
from python.sampling import get_cov_and_mean_rc_theta_32
#from python.conditional_sampling import x_all
#from python.conditional_sampling import x_cond

In [29]:
run_simulation = False

In [25]:
# Set simulating variables.
disc_fac = 0.9999
num_buses = 50
num_periods = 120
gridsize = 1000
# We use the cost parameters and transition probabilities from the replication.
params = np.array([10.07780762, 2.29417622])
trans_probs = np.array([0.39189182, 0.59529371, 0.01281447])
scale = 1e-3

init_dict_simulation = {
    "simulation": {
        "discount_factor": disc_fac,
        "periods": num_periods,
        "seed": 123,
        "buses": num_buses,
    },
    "plot": {"gridsize": gridsize},
}

In [26]:
# Calcualte objects necessary for the simulation process. See documentation for details.
num_states = 200
costs = calc_obs_costs(num_states, lin_cost, params, scale)

trans_mat = create_transition_matrix(num_states, trans_probs)
ev = calc_fixp(trans_mat, costs, disc_fac)[0]

In [27]:
# Use one init_dict for get_demand() and estimate().
init_dict_estimation = {
    'model_specifications': {
        'discount_factor': disc_fac,
        'number_states': num_states,
        'maint_cost_func': 'linear',
        'cost_scale': 1e-3
    },
    'optimizer': {
        'approach': 'NFXP',
        'algorithm': 'scipy_L-BFGS-B',
        'gradient': 'Yes'
    },
    
}

In [28]:
%%time
if run_simulation == True:
    num_sim = 1000
    cov, mean = get_cov_and_mean_rc_theta_32(num_sim, ev, costs, trans_mat, init_dict_simulation, init_dict_estimation)
    np.save(os.path.join('data', 'cov_rc_theta_11'), cov)
    np.save(os.path.join('data', 'mean_rc_theta_11'), mean)
else:
    cov = np.load(os.path.join('data', 'cov_rc_theta_11.npy'))
    mean = np.load(os.path.join('data', 'mean_rc_theta_11.npy'))

Define functions `rust_model()`, and function arguments.

In [32]:
def x_all(n):
    distribution = cp.MvNormal(mean, cov)
    return distribution.sample(n)

def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(subset_j, axis=1)
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(
            n,
            mean=mean,
            cov=cov,
            dependent_ind=subset_j,
            given_ind=subsetj_conditional,
            x_given=xjc,
        )

In [70]:
n_evals = 100
data = x_all(n_evals).T

In [49]:
def get_demand_inputs(x):
    demand_inputs = np.zeros((n_evals, 5))
    demand_inputs[:, :3] = np.array([0.39189182, 0.59529371, 0.01281447])
    demand_inputs[:, 3:] = x[:, :]
    return demand_inputs

In [72]:
demand_inputs = get_demand_inputs(data)

In [118]:
%%time
demand_output = np.zeros((n_evals, 1))
for sample in np.arange(n_evals):
    demand_params = demand_inputs[sample, :]
    demand_output[sample] = get_demand(init_dict_estimation, demand_dict, demand_params).iloc[0]['demand']

Wall time: 40.5 s


In [117]:
%%time
demand_output_list_compr = [get_demand(init_dict_estimation, demand_dict, demand_inputs[sample, :]).iloc[0]['demand']
                            for sample in np.arange(n_evals)]

Wall time: 37.8 s


In [86]:
get_demand_partial = partial(get_demand, init_dict=init_dict_estimation)
get_demand_partial = partial(get_demand_partial, demand_dict=demand_dict)
#map(function, iterable)

In [89]:
get_demand_partial(demand_params=demand_inputs[0])

,demand,success
RC,,
11.5,43.0134,Yes


In [116]:
get_demand_partial = partial(get_demand, init_dict=init_dict_estimation, demand_dict=demand_dict)
def get_demand_mapping(x):
    #get_demand_partial = partial(get_demand_partial, demand_dict=demand_dict)
    out = get_demand_partial(demand_params=x).iloc[0]['demand']
    return out

Wall time: 0 ns


In [115]:
%%time
list(map(get_demand_mapping, demand_inputs))

Wall time: 34.7 s


[43.01337839214233,
 59.04615849320193,
 56.183559210020334,
 63.97994426339406,
 58.9112263777054,
 60.43876110996574,
 51.82971977452244,
 59.201502230398205,
 58.771656243285726,
 59.539581906463006,
 58.8852987524915,
 63.67799814888582,
 60.345296742720066,
 39.11401297286789,
 64.6085420513871,
 70.15448936669314,
 67.2185367977614,
 76.38662930481492,
 71.24314703492456,
 66.00218360123907,
 55.31713314830377,
 64.14175702919518,
 64.83954523980628,
 59.144430971401185,
 65.45647855720176,
 53.46387972651931,
 60.63203100710849,
 71.86696469544658,
 57.318410279066676,
 59.06451406022302,
 65.21560557389397,
 61.46654326377617,
 59.4466568912569,
 56.507258713675554,
 54.30928017804346,
 69.45729526709995,
 58.911877148599686,
 63.66982651190331,
 46.53134291282277,
 59.32451445741021,
 60.92325853013037,
 62.649691414710276,
 70.81519820886129,
 64.24983075960459,
 63.781953169526005,
 48.900439874916415,
 63.94411642993545,
 67.77906173567472,
 62.85851888630546,
 59.094115203

In [67]:
demand_output_list_compr

[61.88918492299763,
 62.338976424144505,
 54.435571086065096,
 56.17475895557953,
 70.34913289445558,
 64.57282897188823,
 57.438442998666844,
 68.82294219853483,
 57.808132649960825,
 65.28633848891201]

In [95]:
def rust_model(x):
    if method == 'exact':
        n_evaluations = n_output + np.math.factorial(n_inputs) * (n_inputs -1) * n_outer * n_inner
    elif method == 'random':
        n_evaluations = n_output + n_perms * (n_inputs -1) * n_outer * n_inner
    demand_inputs = np.zeros((n_evaluations, 5))
    demand_inputs[:, :3] = np.array([0.39189182, 0.59529371, 0.01281447])
    demand_inputs[:, 3:] = x[:, :]
    
    #demand_output = np.zeros((n_evaluations, 1))
    
    demand_output = [get_demand(init_dict_estimation, demand_dict, demand_inputs[sample, :]).iloc[0]['demand'] 
                    for sample in np.arange(n_evaluations)]
    
    #for sample in np.arange(n_evaluations):
    #    demand_params = demand_inputs[sample, :]
    #    demand_output[sample] = get_demand(init_dict_estimation, demand_dict, demand_params).iloc[0][0]
        
    return demand_output

In [58]:
# Need demand at certain value of RC only.  Note RC is scaled by 1e-03.
demand_dict = {
    "RC_lower_bound": 11.5,
    "RC_upper_bound": 11.5,
    "demand_evaluations": 1,
    "tolerance": 1e-10,
    "num_periods": num_periods,
    "num_buses": num_buses,
}

In [41]:
n_inputs = 2
method = 'exact'
np.random.seed(1234)

if method == 'exact':
    n_perms = None
elif method == 'random':
    n_perms = 10
n_output = 10**3
n_outer = 15
n_inner = 3

In [ ]:
def estim_shapley_rc_theta_11(method, n_perms, n_output, n_outer, n_inner, cov, mean):
    
    def x_all(n):
        distribution = cp.MvNormal(mean, cov)
        return distribution.sample(n)

    def x_cond(n, subset_j, subsetj_conditional, xjc):
        if subsetj_conditional is None:
            cov_int = np.array(cov)
            cov_int = cov_int.take(subset_j, axis=1)
            cov_int = cov_int[subset_j]
            distribution = cp.MvNormal(mean[subset_j], cov_int)
            return distribution.sample(n)
        else:
            return _r_condmvn(
                n,
                mean=mean,
                cov=cov,
                dependent_ind=subset_j,
                given_ind=subsetj_conditional,
                x_given=xjc,
            )
    

In [99]:
%%time

exact_shapley = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
exact_shapley.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)

Wall time: 6min 47s


In [106]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.449098,0.31756,-0.173320,1.071515
theta_11,0.550902,0.31756,-0.071515,1.173320
